In [ ]:
!pip install transformers
! pip install umap-learn
! pip install hdbscan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 103.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.4-py3-none-any.whl size=86770 sha256=2641d2c1c77e209ace424400450db41428d8271fb4715f47e6999235a167e510
  Stored in directory: /root/.cache/pip/wheels/fb/66/29/199acf5784d0f7b8add6d466175ab45506c96e386ed5dd0633
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55615 sha256=368fb91a0096c85c169c0af9d47df5541fff9d7a9025dac2161ed6015a9e60e4
  Stored in 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import BertTokenizer, BertModel
import torch
import umap.umap_ as umap
import hdbscan
import torch

In [ ]:
import requests
from io import BytesIO
from zipfile import ZipFile

# Define the URL of the zip file
zip_url = "https://github.com/alberto2020china/fake_jobs/raw/main/data/fake_job_postings.csv.zip"

# Send a GET request to the URL
response = requests.get(zip_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Extract the content of the zip file
    with ZipFile(BytesIO(response.content)) as zip_file:
        # Specify the file you want to extract from the zip archive
        file_to_extract = "fake_job_postings.csv"

        # Extract the file and load it into a Pandas DataFrame (assuming it's a CSV file)
        with zip_file.open(file_to_extract) as file:
            # Now you can use the file (e.g., load it into a Pandas DataFrame)
            # For example:
            # import pandas as pd
            df_raw = pd.read_csv(file)
            print(f"{file_to_extract} has been successfully loaded.")
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")


fake_job_postings.csv has been successfully loaded.


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd

# Assuming df_raw is your original DataFrame
X = df_raw.drop('fraudulent', axis=1)
y = df_raw['fraudulent']

# Define the undersampler
undersampler = RandomUnderSampler(sampling_strategy='majority', random_state=42)

# Perform undersampling
X_resampled, y_resampled = undersampler.fit_resample(X, y)

# Combine the undersampled data back into a DataFrame
df_resampled = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='fraudulent')], axis=1)

In [ ]:
df_resampled = df_resampled.fillna(" ")

In [ ]:
df_resampled['text'] = df_raw['title'] + ' ' + df_raw['location'] + ' ' + df_raw['company_profile'] + ' ' + df_raw['description'] + ' ' + df_raw['requirements'] + ' ' + df_raw['benefits']


In [ ]:
df_resampled['text_1'] = df_raw['title'] + ' ' + df_raw['company_profile'] + ' ' + df_raw['description'] + ' ' + df_raw['requirements'] + ' ' + df_raw['benefits']


In [ ]:
df_resampled['text_2'] = df_raw['company_profile'] + ' ' + df_raw['description'] + ' ' + df_raw['requirements']


In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
import umap
import hdbscan
import pandas as pd

def preprocess_text(text):
    preprocessed_text = str(text).lower()  # Basic preprocessing, modify as needed
    return preprocessed_text

def process_batch(tokenizer, model, jobs):
    tokenized_articles = [tokenizer.encode_plus(
        preprocess_text(job),
        add_special_tokens=True,
        max_length=512,  # Define your desired max length
        padding='max_length',
        truncation=True,
        return_attention_mask=True,  # Include attention mask
        return_tensors='pt'
    ) for job in jobs]

    input_ids = torch.cat([article['input_ids'] for article in tokenized_articles], dim=0)
    attention_masks = torch.cat([article['attention_mask'] for article in tokenized_articles], dim=0)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_masks)
        embeddings = outputs[0][:, 0, :].cpu().numpy()  # Extract the [CLS] token embeddings, move to CPU

    return embeddings

def clustering(df, variable,batch_size=32):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')
    model.eval()

    embeddings_list = []
    for i in range(0, len(df), batch_size):
        batch_articles = df[variable].iloc[i:i+batch_size].tolist()
        batch_embeddings = process_batch(tokenizer, model, batch_articles)
        embeddings_list.append(batch_embeddings)

    embeddings = np.concatenate(embeddings_list, axis=0) if embeddings_list else np.array([])

    if embeddings.size > 0:
        reducer = umap.UMAP(n_components=10)
        reduced_embeddings = reducer.fit_transform(embeddings)

        clusterer = hdbscan.HDBSCAN(min_cluster_size=300)
        cluster_labels = clusterer.fit_predict(reduced_embeddings)

        df['cluster_type'] = cluster_labels
        df['embeddings'] = embeddings.tolist()

    return df


In [ ]:
clustering(df_resampled,"text_2")

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

(…)bert-base-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)base-uncased/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)rt-base-uncased/resolve/main/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,...,required_experience,required_education,industry,function,fraudulent,text_2,text_1,text,cluster_type,embeddings
0,5231,SEM Coordinator,"GR, I, Athens",,,#URL_c379aa631173ed5b7c345ab3f500a9a053e509138...,The right candidate will be responsible for co...,- Previous experience of minimum 3 years...,- Attractive Remuneration Package.- ...,0,...,,,,,0,"We're Food52, and we've created a groundbreaki...",NaN,NaN,1,"[-0.400995671749115, 0.0035485976841300726, -0..."
1,14113,Senior Data Scientist,"GB, LND, London",Engineering,,"As a growing and successful startup, Conversoc...",Conversocial builds software that helps compan...,Good working knowledge of Python or similar la...,Salary of £40-55k plus stock options£2k annual...,0,...,,,,Engineering,0,"90 Seconds, the worlds Cloud Video Production ...",Customer Service - Cloud Video Production 90 S...,"Customer Service - Cloud Video Production NZ, ...",1,"[0.05061265826225281, 0.0749434158205986, 0.32..."
2,3169,Junior Web Marketing Specialist,"GR, I, Ilion",Web Marketing και Social,,Η Atnet Communications A.E. είναι μια αμιγώς ε...,Η Atnet Communications αναζητά Junior Web Mark...,Απαιτούμενα προσόντα:Hands-on εμπειρία στην δι...,H θέση αφορά μόνιμη απασχόληση στην περιοχή το...,0,...,Entry level,,Marketing and Advertising,Marketing,0,Valor Services provides Workforce Solutions th...,NaN,NaN,1,"[-0.4488658010959625, -0.385942280292511, 0.33..."
3,14805,New Product Development Project Leader - Full ...,"US, GA, Atlanta",,,We Provide Full Time Permanent Positions for m...,Coordination and project management of New Pro...,"Location - Atlanta, GA, USAJob Type – Permanen...",,0,...,Mid-Senior level,Bachelor's Degree,Management Consulting,Management,0,Our passion for improving quality of life thro...,Account Executive - Washington DC Our passion ...,"Account Executive - Washington DC US, DC, Wash...",1,"[-0.8715794682502747, -0.04651720076799393, -0..."
4,5810,Data Intern - Retail & Apparel Analysis,"GB, LND, London",Fashion:Internships,14000-14000,"We build software for fashion retailers, to he...",About EDITDEDITD’s software is the market lead...,,,0,...,Internship,,Apparel & Fashion,,0,SpotSource Solutions LLC is a Global Human Cap...,Bill Review Manager SpotSource Solutions LLC i...,"Bill Review Manager US, FL, Fort Worth SpotSou...",1,"[-0.5350897312164307, 0.08209509402513504, -0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1727,17828,Student Positions Part-Time and Full-Time.,"US, CA, Los Angeles",,,,Student Positions Part-Time and Full-Time.You ...,,,0,...,,,,,1,"Shyp is the easiest way to send anything, anyw...",General Manager - Boston Shyp is the easiest w...,"General Manager - Boston US, MA, Boston Shyp i...",1,"[-0.5506725311279297, -0.057693563401699066, 0..."
1728,17829,Sales Associate,"AU, NSW, Sydney",Sales and Marketing,,,LEARN TO EARN AN EXECUTIVE LEVEL INCOMEFULL TR...,What You Can Do. • Have the potential to earn ...,Who We Are We are a Global Leadership Developm...,0,...,,,,,1,Priority Care Group are a family run care grou...,Registered Nurse Priority Care Group are a fam...,"Registered Nurse GB, ABD, Aberdeenshire Priori...",1,"[-0.11531567573547363, 0.0275043286383152, 0.4..."
1729,17830,Android Developer,"PL, MZ, Warsaw",,,,inFullMobile Sp. z o.o. is a mobile software d...,• A proven track record in Android / JAVA proj...,attractive salary (adequate to the candidate s...,0,...,,,,,1,Mindscape is a Wellington based software devel...,Web Designer Mindscape is a Wellington based s...,"Web Designer NZ, N, Wellington Mindscape is a ...",1,"[-0.17596261203289032, -0.06071629747748375, -..."
1730,17831,Payroll Clerk,"US, NY, New York",,,,JOB DESCRIPTIONWe are seeking a full time payr...,JOB REQUIREMENTS• High school diploma or eq...,We offer a competitive salary and benefits pac...,0,...,,,Apparel & Fashion,Accounting

In [ ]:
df_resampled['text'] = df_resampled['text'].astype(str)
print('clustering agg',df_resampled['cluster_type'].value_counts())

clustering agg  1    1147
 0     491
-1      94
Name: cluster_type, dtype: int64


#Topic Modelling

In [ ]:
docs_per_topic_agg = df_resampled.groupby(['cluster_type'], as_index = False).agg({'text': ' '.join})

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count

In [ ]:
def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names_out()
    labels = list(docs_per_topic.cluster_type)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['cluster_type'])
                     .text
                     .count()
                     .reset_index()
                     .rename({"Topic": "cluster", "text": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

In [ ]:
tf_idf_agg, count_agg = c_tf_idf(docs_per_topic_agg.text.values, m=len(df_resampled))
top_n_words_agg = extract_top_n_words_per_topic(tf_idf_agg, count_agg, docs_per_topic_agg, n=20)
topic_sizes_agg = extract_topic_sizes(df_resampled); topic_sizes_agg.head(10)

,cluster_type,Size
2,1,1147
1,0,491
0,-1,94


In [ ]:
cross_tab = pd.crosstab(df_resampled['cluster_type'], df_resampled['fraudulent'])

print(cross_tab)

fraudulent      0    1
cluster_type          
-1             85    9
 0            203  288
 1            578  569


In [ ]:
# Cluster 0
top_n_words_agg[-1]

[('abroad', 0.06703253910763794),
 ('jobs', 0.06485479586614515),
 ('airfare', 0.05015646417361576),
 ('furnished', 0.0496081497347714),
 ('necessarycanada', 0.048701589868479236),
 ('holders', 0.047812429880342656),
 ('usd', 0.04675397623050148),
 ('tesol', 0.046607873803118774),
 ('celta', 0.046607873803118774),
 ('passport', 0.046607873803118774),
 ('tefl', 0.046319286436214274),
 ('reimbursedexcellent', 0.04583972816290506),
 ('teachers', 0.045756039584707434),
 ('teacher', 0.04561802196329796),
 ('kids', 0.045481086844179),
 ('200', 0.04521039710896713),
 ('url_75db76d58f7994c7db24e8998c2fc953ab9a20ea9ac948b217693963f78d2e6b',
  0.04497663336270092),
 ('url_ed9094c60184b8a4975333957f05be37e69d3cdb68decc9dd9a4242733cfd7f7',
  0.04497663336270092),
 ('cardsgabriel', 0.04497663336270092),
 ('adkins', 0.04497663336270092)]

SEMANTIC INTERPRETATION Cluster 0

International Scope: The mention of "abroad," "airfare," and "passport" suggests that these jobs are located outside one's home country, potentially requiring travel and legal documentation.

Educational Focus: Terms like "teaching," "teachers," "teacher," and "kids" point to roles involving education and interaction with children or students. This indicates that the jobs are primarily related to teaching in an educational setting.

Language Teaching: The presence of qualifications like "TESOL," "CELTA," and "TEFL" signifies a focus on teaching English as a foreign language. These certifications suggest specialized language teaching roles.

Benefits and Requirements: Terms like "furnished," "reimbursedexcellent," and "USD" imply potential benefits, including furnished accommodations and excellent reimbursement packages, possibly in U.S. dollars. Additionally, terms like "holders" and "necessarycanada" hint at specific requirements or qualifications, potentially related to teaching in Canada.

Specific Individuals or References: The inclusion of names like "Cardsgabriel" and "Adkins" along with URLs might suggest references to specific individuals, organizations, or job postings relevant to these teaching opportunities.

In essence, these jobs seem to involve teaching English abroad, potentially with specific emphasis on positions requiring certified language instructors (TESOL, CELTA, TEFL) and offering various benefits such as furnished accommodations and financial reimbursements. The cluster also hints at potential requirements specific to teaching in certain locations, possibly Canada, and includes references to individuals or job listings through names and URLs.

In [ ]:
# Cluster 1
top_n_words_agg[1]

[('people', 0.0030477993170439275),
 ('design', 0.0030463070902128487),
 ('technology', 0.003046003567408013),
 ('product', 0.0030457475022242723),
 ('services', 0.0030456023007452275),
 ('customer', 0.0030422833429086084),
 ('development', 0.0030353749704243407),
 ('years', 0.003032408386867887),
 ('clients', 0.0030318963711331057),
 ('marketing', 0.003030739303310015),
 ('ability', 0.003028648113925565),
 ('looking', 0.003027940363103757),
 ('service', 0.003014189194049808),
 ('management', 0.0030121097868927166),
 ('web', 0.0030061036360212),
 ('knowledge', 0.00300525406869251),
 ('working', 0.0029950930954564386),
 ('environment', 0.00298531318947081),
 ('customers', 0.002981052770621869),
 ('including', 0.0029732337237665797)]

SEMANTIC INTERPRETATION CLUSTER 1

The terms provided seem to indicate a cluster related to a broader professional context, potentially in fields like technology, product development, marketing, and customer service. Here's an analysis of the semantic context:

Professional Skills and Abilities: Terms like 'design,' 'technology,' 'product,' 'development,' 'marketing,' 'management,' 'web,' and 'knowledge' suggest a professional environment that could encompass tech-related roles, product development, marketing strategies, and web-based services.

Client/Customer Focus: 'Clients,' 'customers,' 'customer,' 'services,' and 'service' indicate a customer-centric approach, possibly roles that involve providing services or products to clients/customers.

Experience and Years: The mention of 'years' might refer to years of experience, indicating that these job roles might require a certain level of professional experience.

Working Environment: 'Working environment,' 'environment,' and 'including' imply a focus on a conducive work atmosphere or the incorporation of various aspects into the professional sphere.

Ability and Skills: 'Ability' and 'ability to,' along with terms like 'working,' 'including,' and 'knowledge,' suggest a requirement for specific skills or competencies in this professional setting.

In essence, this cluster appears to represent job descriptions or opportunities in a professional context, potentially in technology, product development, marketing, or customer service. It emphasizes skills, experience, a customer-centric approach, and a conducive working environment. The specific terms reflect a diverse set of skills and abilities required in these roles, potentially in a corporate or tech-oriented setting.